In [70]:
from flask import Flask, request, jsonify
from langchain_core.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain_openai import ChatOpenAI
from flask_cors import CORS

import csv

import requests
import threading

app = Flask(__name__)
CORS(app)

# 타로카드 라우트
@app.route("/api/interpret",methods=["POST"])

def tarot_card():
    data = request.json
    cards = data.get("cards")
    reason = data.get("reason")
    user_input = data.get("user_input")
    user_select = data.get("user_select")
    print("recived data: ",data)
    if not cards or len(cards) != 3:
        return jsonify({"error": "카드를 다시 선택해 주세요."}),400
    
    class TarotBot:
        def __init__(self, api_key_path):
            #OpenAI API 설정
            self.api_key = self.load_api_key(api_key_path)
            self.model = ChatOpenAI(openai_api_key = self.api_key, model="gpt-4o")

        def load_api_key(self, file_path):
            with open(file_path, 'r') as file:
                api_key = file.read().strip()
                return api_key
            
        # 사용자 고민이 있을때 타로 해석
        def interpret_cards(self, user_input, cards):
            try:
                tarot_template_reason = PromptTemplate(
                    template=(
                        "사용자의 질문: '{user_input}'\n"
                        "타로카드: {cards}\n"
                        "{cards}의 의미는 한줄로 간략히 출력하고, {user_input}를 주제로 종합한 해석은 최대 500자 이내로 출력해주세요."
                    ),
                    input_variables=["user_input", "cards"]
                )

                reason_chain = LLMChain(llm=self.model, prompt_template=tarot_template_reason)
                reason_response = reason_chain.run(input=user_input, cards=cards)
                answer = reason_response.strip()
                
                if len(answer) > 500:
                    answer = answer[:500]

                return jsonify({"answer": answer})

            except Exception as e:
                return "해석 중 오류가 발생했습니다."
            
        # 일반적인 타로 해석
        def general_reading(self, user_select, cards):
            cards_str = ", ".join(cards)
            try:
                tarot_template_genal = PromptTemplate(
                    template=(
                        "사용자 선택 카테고리: '{user_select}'\n"
                        "타로카드: {cards}\n"
                        "{cards}의 의미는 한줄로 간략히 출력하고, {user_select}를 주제로 종합한 해석은 최대 500자 이내로 출력해주세요.\n"
                    ),
                    input_variables=["user_select", "cards"]
                )

                general_chain = LLMChain(llm=self.model, prompt=tarot_template_genal)
                general_response = general_chain.invoke({"user_select":user_select,"cards":cards_str})
                
                if isinstance(general_response, str):
                    answer = general_response.strip()
                else:
                    answer = general_response
                
                if len(answer) > 500:
                    answer = answer[:500]

                
                print("응답내용",answer)

                return jsonify({"answer": answer})

            except Exception as e:
                print((f"Error: {e}"))
                return "일반 해석 중 오류가 발생했습니다."
            

    key_path = "C:\\Users\\SMHRD\\Desktop\\ky_api.txt"
    try:
        tarot = TarotBot(api_key_path=key_path)
        if reason:
            response = tarot.interpret_cards(user_input,cards)
        else:
            response = tarot.general_reading(user_select,cards)
        print("response: ",response)
        return response
    
    except Exception as e:
        print("Error",str(e))
        return jsonify({"error":str(e)}),500

    


if __name__ == "__main__":
    app.run(port=7000) #Flask 서버 포트 설정

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:7000
Press CTRL+C to quit
127.0.0.1 - - [02/Sep/2024 16:03:23] "OPTIONS /api/interpret HTTP/1.1" 200 -


recived data:  {'user_select': '오늘의 운세', 'cards': ['Eight of Wands', 'The World', 'The Moon']}


127.0.0.1 - - [02/Sep/2024 16:03:26] "POST /api/interpret HTTP/1.1" 200 -


응답내용 {'user_select': '오늘의 운세', 'cards': 'Eight of Wands, The World, The Moon', 'text': '**카드 의미 한줄 요약:**\n- Eight of Wands: 빠른 진행과 변화\n- The World: 완성과 성취\n- The Moon: 불확실성과 직감\n\n**오늘의 운세 종합 해석:**\n오늘 당신의 하루는 빠르게 진행되는 변화와 성취의 기운이 가득할 것입니다. Eight of Wands는 일이 급속도로 진전될 수 있음을 암시하며, The World는 그 결과가 매우 긍정적일 것임을 예고합니다. 그러나 The Moon이 나타난 것을 보면, 이러한 변화와 성취 속에서 불확실함과 직감에 의존해야 하는 상황도 예상됩니다. 직감에 귀 기울이고, 불확실한 상황에서도 침착함을 유지한다면, 오늘 당신에게 매우 의미 있는 성취를 이룰 수 있는 날이 될 것입니다.'}
response:  <Response 1494 bytes [200 OK]>
